In [8]:
import pandas as pd
import requests
import sqlite3
from typing import Set

# Cargando los datos de la db

In [9]:
def ej_1_cargar_datos_demograficos() -> pd.DataFrame:
    ruta_archivo_demograficos = 'us-cities-demographics.csv'
    datos_demograficos = pd.read_csv(ruta_archivo_demograficos, delimiter=';')
    return datos_demograficos

# Parseando datos de calidad de aire por ciudad

In [10]:
def obtener_calidad_aire(ciudad, estado):
    url_api_calidad_aire = 'https://api-ninjas.com/api/airquality'
    parametros = {'city': ciudad, 'state': estado}
    respuesta = requests.get(url_api_calidad_aire, params=parametros)

    if respuesta.status_code == 200:
        datos_calidad_aire = respuesta.json()
        concentracion = datos_calidad_aire.get('concentration', None)
        return concentracion
    else:
        return None

def ej_2_cargar_calidad_aire(ciudades: Set[str]) -> None:
    datos_demograficos = ej_1_cargar_datos_demograficos()
    datos_demograficos['concentration_air_quality'] = datos_demograficos.apply(
        lambda row: obtener_calidad_aire(row['City'], row['State']) if row['City'] in ciudades else None, 
        axis=1
    )
    tabla_calidad_aire = datos_demograficos[['City', 'State', 'concentration_air_quality']]
    tabla_calidad_aire.to_csv('tabla_calidad_aire.csv', index=False)


# Limpieza de datos demográficos

In [11]:
def ej_3_limpiar_datos_demograficos(datos_demograficos: pd.DataFrame) -> pd.DataFrame:
    # Eliminar columnas
    datos_demograficos = datos_demograficos.drop(['Race', 'Count', 'Number of Veterans'], axis=1)
    
    # Eliminar filas duplicadas
    datos_demograficos = datos_demograficos.drop_duplicates()
    
    return datos_demograficos

#  Crear base de datos SQLite y cargar las tablas

In [12]:
def ej_4_crear_base_de_datos(datos_demograficos: pd.DataFrame, tabla_calidad_aire: pd.DataFrame) -> None:
    conn = sqlite3.connect('demografia_calidad_aire.db')
    datos_demograficos.to_sql('demografia', conn, index=False, if_exists='replace')
    tabla_calidad_aire.to_sql('calidad_aire', conn, index=False, if_exists='replace')
    conn.close()

# Aplicar joins y agregaciones

In [13]:
def ej_5_verificar_calidad_aire(ciudades_mas_pobladas: int) -> pd.DataFrame:
    conn = sqlite3.connect('demografia_calidad_aire.db')

    # Realizar join entre las dos tablas
    query = f'''
    SELECT d.City, d.State, d.TotalPopulation, ca.concentration_air_quality
    FROM demografia d
    LEFT JOIN calidad_aire ca ON d.City = ca.City AND d.State = ca.State
    ORDER BY d.TotalPopulation DESC
    LIMIT {ciudades_mas_pobladas}
    '''
    resultado = pd.read_sql_query(query, conn)

    # Mostrar las primeras 10 columnas
    resultado = resultado.iloc[:, :10]

    conn.close()

    return resultado
